In [1]:
# code
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import gc
import psutil
import multiprocessing
from multiprocessing import Pool
import time
# for multiprocessing on iPython,
# workers need to be in separate modules
from xpu_workers import make_dots


# due to how multiprocessing works with iPython and/or Windows
# we need to wrap the main code under __main__
# this would not be necessary with pure Python on Unix
if __name__ == '__main__':
    tstart = int(round(time.time() * 1000))
    # set variables
    d_max = 31
    
    # The more points we use, the bigger the RAM usage.
    # Instead of increasing the number of points,
    # just loop the same algorithm and summarize the results.
    # Effective points is: points * pointloops
    points = 10 ** 7
    pointloops = 10
    
    dims = []
    ratios = []
    
    # figure out system
    try:
        import cupy as cp
        gpumem = cp.cuda.Device().mem_info[1]
    except:
        gpumem = 0
    num_p = multiprocessing.cpu_count()
    sysmem = psutil.virtual_memory()[0]
    memGB = round(sysmem / (1024 ** 3))
    print('sys:', num_p, 'CPUs,', memGB, 'GB RAM')
    print('GPU:', round(gpumem / (1024 ** 3)), 'GB')
    print("Calculating with %g points" % (points * pointloops))
    print()
    
    # initialize plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.set_xlabel('N (# of dimensions)')
    ax1.set_ylabel('ratio')
    ax1.set_title('sphere/cube volume ratio')
    ax1.set_ylim(0, 0.8)
    ax1.set_xscale('linear')
    ax1.set_yscale('linear')
    ax1.set_xticks(range(2, d_max))
    ax1.grid()
    square1 = plt.Rectangle((-0.5, -0.5), 1, 1, linewidth = 1, edgecolor='black', facecolor='white')
    circle1 = plt.Circle((0, 0), 0.5, color='black', fill=False)
    
    if gpumem > 0:
        num_p = 1
    print('dims\tratios')
    # main loop
    for d in range(2, d_max):
        # create MP pool
        p = Pool(processes = num_p)
        
        # multiprocessing.pool.map() only works with one argument,
        # so we make a list of tuples
        # each tuple is an "argument" to a worker
        arglist = [(points, d, num_p, sysmem, gpumem, pointloops)] * num_p
        # Wake up comrades! For the glory of the Motherland!
        work_out = p.map(make_dots, arglist)
        # In Jupyter, if you don't force close the pool every cycle,
        # you get A LOT of workers just hanging around
        # and the whole thing stops. Kinda like the 1930s.
        p.close()
        
        # parse the output from workers
        p_int = 0
        pts_sample = np.empty((0, d))
        for i in range(num_p):
            p_int += work_out[i][0]
            pts_sample = np.concatenate((pts_sample, work_out[i][1]))
        ratio = p_int / points
        dims.append(d)
        ratios.append(ratio)
        
        # plot the ratios
        ax1.set_xlim(2, max(d, 3))
        ax1.plot(dims, ratios, '.b-')
        
        # plot the sample dots
        ax2.clear()
        ax2.set_aspect(1.0)
        ax2.set_xscale('linear')
        ax2.set_yscale('linear')
        ax2.set_axis_off()
        ax2.set_xlim(-0.51, 0.51)
        ax2.set_ylim(-0.51, 0.51)
        ax2.set_title('Monte Carlo simulation:\nprojection of random sample of points\non thin central cross-section')
        ax2.add_artist(square1)
        ax2.add_artist(circle1)
        ax2.plot(pts_sample[:, 0], pts_sample[:, 1], '.r')
        del pts_sample
        
        # update the figure (actually plots on screen)
        fig.canvas.draw()
        print("%d\t%g" % (d, ratio))
        gc.collect()
        if p_int == 0:
            break
    tend = int(round(time.time() * 1000))
    print('\nExecution time:', (tend - tstart) / 1000, 'sec')


sys: 12 CPUs, 16 GB RAM
GPU: 6 GB
Calculating with 1e+08 points



<IPython.core.display.Javascript object>

dims	ratios
2	0.785381
3	0.523674
4	0.308478
5	0.164479
6	0.0807407
7	0.036932
8	0.015822
9	0.00644278
10	0.00248438
11	0.00092238
12	0.00032718
13	0.00011231
14	3.616e-05
15	1.124e-05
16	3.53e-06
17	1.01e-06
18	3.9e-07
19	9e-08
20	1e-08
21	0

Execution time: 56.473 sec
